In [ ]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteOYL7GL/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteOYL7GL/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteOYL7GL/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
for item in imdb['train']:
  print(item)
  break

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [ ]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s, l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

for s, l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [ ]:
vocab_size = 10000 #총 word_index는 8만갠데 쓰는 것은 만개
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token= oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen = max_length, truncating=trunc_type)


In [ ]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences)

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [ ]:
reverse_word_index[1]

'<OOV>'

In [ ]:
len(word_index)

88583

In [ ]:
def decode_review(text):
  return ' '.join([reverse_word_index.get(i,'?') for i in text])

print(decode_review(padded[3]))
# 0에 대한 index가 없기 때문에 ?이고, 사전에 없으면 oov로 나옴

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received


In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             tf.keras.layers.GlobalAveragePooling1D(),
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid') # 1이나 아니냐->dense1이고 이진분류니까 sigmoid
])
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - ETA: 0s - loss: 0.5642 - accuracy: 0.7574WARNING:tensorflow:Model was constructed with shape (None, 120) for input Tensor("embedding_1_input:0", shape=(None, 120), dtype=float32), but it was called on an input with incompatible shape (None, 2332).


782/782 [==============================] - 5s 7ms/step - loss: 0.5642 - accuracy: 0.7574 - val_loss: 0.7269 - val_accuracy: 0.5032
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3432 - accuracy: 0.8592 - val_loss: 0.6383 - val_accuracy: 0.5432
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2784 - accuracy: 0.8880 - val_loss: 0.6282 - val_accuracy: 0.5555
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2421 - accuracy: 0.9042 - val_loss: 0.6082 - val_accuracy: 0.6058
Epoch 5/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2162 - accuracy: 0.9195 - val_loss: 0.6013 - val_accuracy: 0.6138
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1946 - accuracy: 0.9287 - val_loss: 0.6923 - val_accuracy: 0.5221
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1774 - accuracy: 0.9353 - val_loss: 0.5793 - val_accuracy: 0.6749
Epoch 8/10
782/78

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)
# 10000이 vocab, 16이 embeddin size

(10000, 16)


In [ ]:
import io

out_v = io.open('vecs.tsc','w',encoding='utf-8')
out_m = io.open('mets.tsc','w',encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word +'\n')
  out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')
out_v.close()
out_m.close()